In [1]:
#### Convert coffea output into root TH2's to feed into TUnfold
import coffea
import pickle
import uproot
import awkward as ak
import matplotlib.pyplot as plt
import numpy as np
import os
# import ROOT

In [4]:
from plugins import addFiles
files = ["coffeaOutput/trijetHists_JetHT_pt200.0_rappidity2.5_bbloosejesjec2018.pkl", "coffeaOutput/trijetHists_JetHT_pt200.0_rappidity2.5_bbloosejesjec2017.pkl", "coffeaOutput/trijetHists_JetHT_pt200.0_rappidity2.5_bbloosejesjec2016.pkl"]
#files = ["coffeaOutput/trijetHists_wXSscaling_QCDsim_pt200.0rapidity2.5_bbloosejesjec2018.pkl", "coffeaOutput/trijetHists_wXSscaling_QCDsim_pt200.0rapidity2.5_bbloosejesjec2017.pkl", "coffeaOutput/trijetHists_wXSscaling_QCDsim_pt200.0rapidity2.5_bbloosejesjec2016.pkl"]
combinedFile = addFiles(files)
print(combinedFile)




coffeaOutput/trijetHists_JetHT_pt200.0_rappidity2.5_bbloosejesjecALL.pkl


In [9]:
%matplotlib inline
fname = "coffeaOutput/trijetHists_wXSscaling_QCDsim_pt200.0rapidity2.5_bbloosejesjecALL.pkl"
# fname = "coffeaOutput/trijetHists_wXSscaling_QCDsim_pt200.0rapidity2.5_bbloosejesjec2018.pkl"
# fname = "coffeaOutput/trijetHists_JetHT_pt200.0_rappidity2.5_bbloosejesjec2018.pkl"
# fname ="coffeaOutput/trijetHistsTest_JetHT_pt200.0_eta2.4_bbloose.pkl"
eras = ["2016", "2017", "2018", "2016APV"]
dir = "rootFiles/"
for year in eras:
    print(year)
    if year in fname:
        print(year)
        year_str = year
        break
    else:
        year_str = "ALL"
with open(fname, "rb") as f:
    result = pickle.load( f )
axis_names = [ax.name for ax in result['jet_pt_mass_reco_u'].axes]
print(axis_names)
cats = [cat for cat in result['jet_pt_mass_reco_u'][{'ptreco':sum, 'dataset':sum, 'mreco':sum}].axes[0]]
if "JetHT" in fname:
    if "dijet" in  fname:
        rootfname = 'dijetHistsJetHT_jec_' + year_str + '.root'
    elif "trijet" in fname:
        rootfname = 'trijetHistsJetHT_jec_' + year_str + '.root'
elif "QCD" in fname:
    if "dijet" in  fname:
        rootfname = 'dijetHistsQCDsim_jec_' + year_str + '.root'
    # integrate and sum over axes
    elif "trijet" in fname:
        rootfname = 'trijetHistsQCDsim_jec' + year_str + '.root'
if os.path.exists(dir + rootfname):
    rootfile = uproot.recreate(dir+rootfname)
else:
    rootfile = uproot.create(dir+rootfname)
    #### only add hists containing gen if SIM files
response_entries = {}
for syst in cats:
    rootfile['ptreco_mreco_u_'+syst] = result['jet_pt_mass_reco_u'][{'dataset':sum, 'syst':syst}]
    rootfile['ptreco_mreco_g_'+syst] = result['jet_pt_mass_reco_g'][{'dataset':sum, 'syst':syst}]
    if "QCD" in fname:
        rootfile['ptgen_mgen_u_'+syst] = result['response_matrix_u'][{'syst':syst}].project("ptgen", "mgen")
        rootfile['ptgen_mgen_g_'+syst] = result['response_matrix_g'][{'syst':syst}].project("ptgen","mgen")
        rootfile['ptreco_mreco_u_'+syst] = result['response_matrix_u'][{'syst':syst}].project("ptreco", "mreco")
        rootfile['ptreco_mreco_g_'+syst] = result['response_matrix_g'][{'syst':syst}].project("ptreco","mreco")
        rootfile['misses_ptgen_mgen_'+syst] = result['misses'][{'syst':syst}].project("ptgen", "mgen")
        rootfile['fakes_ptreco_mreco_'+syst] = result['fakes'][{'syst':syst}].project("ptreco","mreco")
        response_matrix_u_values, ptreco_edges, mreco_edges, ptgen_edges, mgen_edges = result['response_matrix_u'][{'syst':syst}].project("ptreco", "mreco", "ptgen", "mgen").to_numpy(flow=True)
        response_matrix_g_values = result['response_matrix_g'][{'syst':syst}].project("ptreco", "mreco", "ptgen", "mgen").values(flow=True)
        print(response_matrix_g_values.shape)
        ptreco_centers = (ptreco_edges[:-1]+ptreco_edges[1:])/2
        print(ptreco_centers[np.newaxis])
        mreco_centers = (mreco_edges[:-1]+mreco_edges[1:])/2
        ptgen_centers = (ptgen_edges[:-1]+ptgen_edges[1:])/2
        mgen_centers = (mgen_edges[:-1]+mgen_edges[1:])/2
        print(np.shape(response_matrix_g_values))
        print(np.shape(response_matrix_g_values[np.newaxis]))
        response_entries.update({'groomed_'+syst:response_matrix_g_values[np.newaxis],
                       'ungroomed_'+syst:response_matrix_u_values[np.newaxis]})
        centers = {'ptreco':ptreco_centers[np.newaxis], 'mreco':mreco_centers[np.newaxis], 
                      'ptgen':ptgen_centers[np.newaxis], 'mgen':mgen_centers[np.newaxis]}
        rootfile['response']=response_entries
        rootfile['centers'] = centers
rootfile.close()

2016
2017
2018
2016APV
['dataset', 'syst', 'ptreco', 'mreco']
(12, 26, 12, 14)
[[ -inf  240.  320.  405.  485.  575.  660.  720.  775. 1050. 7150.   inf]]
(12, 26, 12, 14)
(1, 12, 26, 12, 14)
(12, 26, 12, 14)
[[ -inf  240.  320.  405.  485.  575.  660.  720.  775. 1050. 7150.   inf]]
(12, 26, 12, 14)
(1, 12, 26, 12, 14)
(12, 26, 12, 14)
[[ -inf  240.  320.  405.  485.  575.  660.  720.  775. 1050. 7150.   inf]]
(12, 26, 12, 14)
(1, 12, 26, 12, 14)
(12, 26, 12, 14)
[[ -inf  240.  320.  405.  485.  575.  660.  720.  775. 1050. 7150.   inf]]
(12, 26, 12, 14)
(1, 12, 26, 12, 14)
(12, 26, 12, 14)
[[ -inf  240.  320.  405.  485.  575.  660.  720.  775. 1050. 7150.   inf]]
(12, 26, 12, 14)
(1, 12, 26, 12, 14)


In [10]:
response_matrix_u_values, ptreco_edges, mreco_edges, ptgen_edges, mgen_edges = result['response_matrix_u'][{'syst':syst}].project("ptreco", "mreco", "ptgen", "mgen").to_numpy(flow=True)
print(response_matrix_u_values.shape)
response_matrix_u_values= result['response_matrix_u'][{'syst':syst}].project("ptreco", "mreco", "ptgen", "mgen").values()
ptreco_mreco = result['response_matrix_u'].project("ptreco", "mreco").values()
nptreco,nmassreco,nptgen,nmassgen = response_matrix_u_values.shape
response_matrix_u_final = response_matrix_u_values.reshape( (nptreco)*(nmassreco), (nptgen)*(nmassgen) ) 
print(response_matrix_u_final.shape)
response_matrix_u_final2 = result['response_matrix_u'][{'syst':"nominal"}].project("ptreco", "mreco", "ptgen", "mgen").values(flow=True).reshape( (nptreco)*(nmassreco), (nptgen)*(nmassgen) ) 
print(response_matrix_u_final2)
print(np.sum(response_matrix_u_final[27,:]))
print(np.sum(response_matrix_u_final2[:,1]))

(12, 26, 12, 14)
(240, 120)


ValueError: cannot reshape array of size 52416 into shape (240,120)

In [ ]:
# fname = uproot.open('root://cmsxrootd.fnal.gov//store/mc/RunIISummer20UL18NanoAODv9/QCD_Pt_1400to1800_TuneCP5_13TeV_pythia8/NANOAODSIM/106X_upgrade2018_realistic_v16_L1v1-v1/280000/42B4DEA0-C0E7-A944-80DC-1D107A0F63FB.root')

In [ ]:
# print(fname["Events"].keys("*HTXS*"))

In [ ]:
hists = uproot.open('rootFiles/trijetHistsJetHT_jec_2018.root')
print(hists.keys())
resp_groomed = hists['response']['groomed_nominal'].array().to_numpy()[0]
print(hists['ptreco_mreco_u_nominal'].to_numpy())
hist_vals, hist_edges, hist_widths = hists['ptreco_mreco_u_nominal'].to_numpy(flow=True)
print(hist_vals[1][1])
histg_vals, histg_edges, histg_widths = hists['ptgen_mgen_u_nominal'].to_numpy(flow=False)
print(histg_vals[0][0])